In [17]:
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
import os
import open3d as o3d

In [18]:
# Enumerate connected RealSense cameras
ctx = rs.context()
devices = ctx.query_devices()
device_serials = [device.get_info(rs.camera_info.serial_number) for device in devices]
device_serials.sort()

# Print selected device serial numbers
print("Selected device serial numbers:", device_serials)

# Create a pipeline for each camera
pipelines = []
profiles = []
align = rs.align(rs.stream.color)

# Initialize RealSense pipeline for each camera
for idx, serial in enumerate(device_serials):
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_device(serial)
    config.enable_stream(rs.stream.color, 1920, 1080, rs.format.rgb8, 30)
    config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
    profile = pipeline.start(config)
    profiles.append(profile)
    pipelines.append(pipeline)

Selected device serial numbers: ['233522072900']


In [19]:
def convert_rs_frames_to_pointcloud(rs_frames, pinhole_camera_intrinsic, extrinsic):
    aligned_frames = align.process(rs_frames)
    rs_depth_frame = aligned_frames.get_depth_frame()
    np_depth = np.asanyarray(rs_depth_frame.get_data())
    o3d_depth = o3d.geometry.Image(np_depth)

    rs_color_frame = aligned_frames.get_color_frame()
    np_color = np.asanyarray(rs_color_frame.get_data())
    o3d_color = o3d.geometry.Image(np_color)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d_color, o3d_depth, depth_scale=1000.0, convert_rgb_to_intensity=False)

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd, pinhole_camera_intrinsic, extrinsic)

    return pcd


In [20]:
cam_2_optical = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]

extrinsic2 = np.eye(4)

extrinsic1 = np.array([[-0.98792383, -0.0497825, 0.14672493, -0.00776776],
                        [0.1269471, -0.80299654, 0.58230662, -0.63564131],
                        [0.08883093, 0.59390089, 0.79961916, 0.74647112],
                        [0, 0, 0, 1]])

extrinsics = [extrinsic2@cam_2_optical, extrinsic1@cam_2_optical]

In [21]:

pcds = []  # 存储点云的列表
for i, pipeline in enumerate(pipelines):
    # 获取相机的内参
    intr = profiles[i].get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
    pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
        intr.width, intr.height, intr.fx, intr.fy, intr.ppx, intr.ppy)
    
    # 捕获帧并转换
    rs_frames = pipeline.wait_for_frames()
    extrinsic = extrinsics[i]
    pcd = convert_rs_frames_to_pointcloud(rs_frames, pinhole_camera_intrinsic, extrinsic)
    pcds.append(pcd)

    if i == 0:
        # pcd to red
        pcd.paint_uniform_color([1, 0, 0])

# 融合两个点云
combined_pcd = pcds[0] + pcds[1]

# # downsample
# voxel_size = 0.002
# combined_pcd = combined_pcd.voxel_down_sample(voxel_size)

# 创建Open3D视窗并显示融合的点云
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Combined Point Cloud", width=800, height=800)
vis.add_geometry(combined_pcd)
render_opt = vis.get_render_option()
render_opt.point_size = 1.0

vis.run()  # 显示窗口
vis.destroy_window()

IndexError: list index out of range

In [ ]:
# # 创建Open3D视窗
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Dynamic Combined Point Cloud", width=800, height=800)
# render_opt = vis.get_render_option()
# render_opt.point_size = 1.0

# if_first = True

# # 循环不断更新显示点云
# while True:
#     pcds = []  # 存储点云的列表
#     for i, pipeline in enumerate(pipelines):
#         # 获取相机的内参
#         intr = profiles[i].get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
#         pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
#             intr.width, intr.height, intr.fx, intr.fy, intr.ppx, intr.ppy)
        
#         # 捕获帧并转换
#         extrinsic = extrinsics[i]
#         rs_frames = pipeline.wait_for_frames()
#         pcd = convert_rs_frames_to_pointcloud(rs_frames, pinhole_camera_intrinsic, extrinsic)
#         if i == 0:
#             # 将第一个相机的点云着色为红色
#             pcd.paint_uniform_color([1, 0, 0])
#         pcds.append(pcd)

#     # 融合两个点云
#     combined_pcd = pcds[0] + pcds[1]

#     # 如果是第一次循环，添加点云到视图中；否则更新点云
#     if if_first:
#         vis.add_geometry(combined_pcd)
#         if_first = False
#     else:
#         vis.update_geometry(combined_pcd)
    
#     vis.poll_events()
#     vis.update_renderer()

#     print(1)

# vis.destroy_window()